In [67]:
import math
import random

In [68]:
# calculate a random number where:  a <= rand < b
def rand(a, b):
    return (b-a)*random.random() + a

# Make a matrix (we could use NumPy to speed this up)
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
#     return math.tanh(x)
    return 1/(1+math.pow(math.e,-x))

# derivative of our sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
#     return 1.0 - y**2
    return y*(1-y)

In [79]:
class NN:
    def __init__(self, ni, nh, no):
        # number of input, hidden, and output nodes
        self.ni = ni + 1 # +1 for bias node
        self.nh = nh
        self.no = no

        # activations for nodes
        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no
        
        # create weights
        self.wi = [[0.15,0.25],[0.2,0.3],[0.5,0.5]]
        self.wo = [[0.4], [0.5]]
#         self.wi = makeMatrix(self.ni, self.nh)
#         self.wo = makeMatrix(self.nh, self.no)
#         # set them to random vaules
#         for i in range(self.ni):
#             for j in range(self.nh):
#                 self.wi[i][j] = rand(-0.2, 0.2)
#         for j in range(self.nh):
#             for k in range(self.no):
#                 self.wo[j][k] = rand(-2.0, 2.0)
        
        # last change in weights for momentum   
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)
    
    ## 計算網路的輸出的函數
    def update(self, inputs):
        if len(inputs) != self.ni-1:
            raise ValueError('wrong number of inputs')

        # input activations
        for i in range(self.ni-1):
            self.ai[i] = inputs[i]

        # hidden activations 計算隱藏層輸出值
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum = sum + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)
#             print('H',j+1,': ',self.ah[j])

        # output activations 計算輸出層輸出值 
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum = sum + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)
#             print('y',j+1,': ',self.ao[k])

        return self.ao[:]
    
    ## backPropagate()：反傳遞學習的函數
    def backPropagate(self, targets, N, M):
        if len(targets) != self.no:
            raise ValueError('wrong number of target values')

        # calculate error terms for output
        output_deltas = [0.0] * self.no
        for k in range(self.no):
#             error = targets[k]-self.ao[k]
            error=(targets[k]-self.ao[k])
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # calculate error terms for hidden
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # update output weights
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
                self.co[j][k] = change

        # update input weights
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
                self.ci[i][j] = change

        # calculate error
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k]-self.ao[k])**2
        return error
    
    def test(self, patterns):
        pred=[]
        for p in patterns:
            print('input:',p[0], '-> predict:', self.update(p[0]),' target:',p[1])
            pred.append(self.update(p[0])[0])
        return pred

    def weights(self):
        print('Input weights:')
        for i in range(self.ni):
            print(self.wi[i])
        print('Output weights:')
        for j in range(self.nh):
            print(self.wo[j])

    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
            if i % 100 == 0:
                print('error %-.6f' % error)

In [80]:
# Tranning data
data = [[[2,-1],[0.01]]]

# create a network with two input, two hidden, and one output nodes
n = NN(2, 2, 1)

# test it
pred=n.test(data)
# train it with some patterns
n.train(data,1,0.5,0.1)
print('--------------------------------------------------')
# 學習後的權重
n.weights()
print('--------------------------------------------------')


input: [2, -1] -> predict: [0.6439056324001806]  target: [0.01]
error 0.200918
--------------------------------------------------
Input weights:
[0.13669857911694533, 0.23388713183993481]
[0.20665071044152736, 0.3080564340800326]
[0.4933492895584727, 0.4919435659199674]
Output weights:
[0.35307726256753064]
[0.45143979995748873]
--------------------------------------------------


In [77]:
# 觀察學習N代後預測
pred=n.test(data)

input: [1, 2] -> predict: [0.9999580834172894]  target: [3]
